In [5]:
import pandas as pd
import numpy as np

from util import comparar_columnas, eliminar_caracteristicas, prueba_umbral
from feature_selection import *

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import joblib
import json
import os

In [6]:
final = pd.read_csv('final_dataframe.csv', index_col=False)
selected_features = ['ksat_0-5cm.tif',
'PP',
'PWP.100-200cm.tif',
'slope',
'Bulkd.5-15cm.tif',
'valor_humedad_suelo1',
'Valor']
final = final[selected_features]

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
import joblib
import os
import json

def entrenar_y_guardar_modelos(df, columna_objetivo, ruta_guardado):
    # Separar características y variable objetivo
    X = df.drop(columns=[columna_objetivo])
    y = df[columna_objetivo]
    
    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    # Definir los modelos y sus hiperparámetros para GridSearchCV
    modelos = {
        'RandomForest': {
            'modelo': RandomForestClassifier(random_state=42),
            'parametros': {
                'classifier__n_estimators': [100, 200, 300, 500],
                'classifier__max_depth': [3, 5, 7, 10, 15, 20, None],
                'classifier__min_samples_split': [2, 5, 10, 15, 20],
                'classifier__min_samples_leaf': [1, 2, 4, 6, 8],
                'classifier__max_features': ['sqrt', 'log2', None],
                'classifier__bootstrap': [True, False],
                'classifier__random_state': [42],
                'classifier__max_leaf_nodes': [None, 50, 100, 200],
                'classifier__min_weight_fraction_leaf': [0.0, 0.1, 0.2],
                'classifier__min_impurity_decrease': [0.0, 0.01, 0.05],
                'classifier__n_jobs': [-1]
            }
        },
        'SVC': {
            'modelo': SVC(probability=True, random_state=42),
            'parametros': {
                'classifier__C': [0.1, 1, 2, 10, 100],
                'classifier__kernel': ['rbf'],
                'classifier__gamma': ['scale', 'auto', 0.01, 0.1, 1, 10, 20, 30, 50, 100],
                'classifier__random_state': [42]
            }
        },
        'XGBoost': {
            'modelo': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
            'parametros': {
                'classifier__max_depth': [3, 5, 7, 9, 11, 13],
                'classifier__min_child_weight': [1, 3, 5, 7],
                'classifier__gamma': [0, 0.1, 0.2, 0.3, 0.4],
                'classifier__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                'classifier__n_estimators': [100, 200, 300, 500],
                'classifier__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                'classifier__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                'classifier__reg_alpha': [0, 0.1, 0.5, 1.0],
                'classifier__reg_lambda': [0.1, 0.5, 1.0, 1.5, 2.0],
                'classifier__scale_pos_weight': [1, 3, 5],
                'classifier__random_state': [42]
            }
        }
    }
    
    metricas = {}
    
    for nombre, item in modelos.items():
        modelo = item['modelo']
        parametros = item['parametros']
        
        # Crear un Pipeline que incluye el escalado y el modelo
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', modelo)
        ])
        
        # Realizar la búsqueda de hiperparámetros
        grid = GridSearchCV(
            pipeline,
            param_grid=parametros,
            cv=3,
            scoring='accuracy',
            n_jobs=-1,
            verbose=1
        )
        grid.fit(X_train, y_train)
        mejor_modelo = grid.best_estimator_
        
        # Guardar el mejor modelo en un archivo .pkl
        nombre_archivo_modelo = os.path.join(ruta_guardado, f"{nombre}_mejor_modelo.pkl")
        joblib.dump(mejor_modelo, nombre_archivo_modelo)
        
        # Predecir en el conjunto de prueba
        y_pred = mejor_modelo.predict(X_test)
        
        # Calcular métricas
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensibilidad = tp / (tp + fn)  # Recall
        especificidad = tn / (tn + fp)  # Specificity
        precision = precision_score(y_test, y_pred)
        exactitud = accuracy_score(y_test, y_pred)
        
        metricas[nombre] = {
            'Sensibilidad': sensibilidad,
            'Especificidad': especificidad,
            'Precisión': precision,
            'Exactitud': exactitud,
            'Mejores Hiperparámetros': grid.best_params_
        }
        
        # Añadir los nombres de las características al modelo
        mejor_modelo.named_steps['classifier'].feature_names = X_train.columns.tolist()
    
    # Guardar las métricas en un archivo .txt
    nombre_archivo_metricas = os.path.join(ruta_guardado, 'metricas_modelos.txt')
    with open(nombre_archivo_metricas, 'w') as file:
        json.dump(metricas, file, indent=4)
    
    return metricas

In [ ]:
target = 'Valor'
path_guardar = 'resultados_ga\ga_randomforest_mutflip_0_3_mutprob_0_3_cpx_0_6'
entrenar_y_guardar_modelos(final, target, path_guardar)

Fitting 3 folds for each of 151200 candidates, totalling 453600 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 3600000 candidates, totalling 10800000 fits
